In [1]:
import boto3
import sagemaker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


In [2]:
s3 = boto3.resource('s3')

In [3]:
for bucket in s3.buckets.all():
    print(bucket.name)

kinesisbucketnugatest
sagemaker-eu-north-1-928389782162
sagemaker-practice-bucket-nuga
sagemaker-studio-v9s6knew9ua
sagemaker-studio-xv6zxh7iw6m
sagemaker-us-east-1-928389782162


In [4]:
with open('/Users/Labi/V_env/ML_Orchestration/data/fitness_class_2212.csv', 'rb') as data:
    s3.Bucket('sagemaker-practice-bucket-nuga').put_object(Key='fitness_class_2212.csv', Body=data)

In [8]:
# CONFIG 
client = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'sagemaker-practice-bucket-nuga'

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


In [14]:
data_path = '/Users/Labi/V_env/ML_Orchestration/data/encoded_data.csv'

In [15]:
data = pd.read_csv(data_path)

In [16]:
data

,day_of_week,time,category_-,category_Aqua,category_Cycling,category_HIIT,category_Strength,category_Yoga,months_as_member,weight,days_before,attended
0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,17,79.56,8,0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10,79.01,2,0
2,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16,74.53,14,0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,86.12,10,0
4,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,15,69.29,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21,79.51,10,0
1496,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,29,89.55,2,0
1497,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,87.38,4,0
1498,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,34,68.64,14,0


In [18]:
X = data.drop('attended', axis=1)
y = data['attended']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [31]:
X_train

,day_of_week,time,category_-,category_Aqua,category_Cycling,category_HIIT,category_Strength,category_Yoga,months_as_member,weight,days_before
382,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,79.07,4
538,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28,72.40,4
1493,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,80.46,8
1112,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,86.34,10
324,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,111.16,10
...,...,...,...,...,...,...,...,...,...,...,...
1130,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15,77.20,10
1294,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17,85.26,6
860,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,6,95.66,3
1459,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,13,73.31,12


In [42]:
pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1).to_csv('train.csv', index=False)

In [43]:
prefix = 'fitness_project/data'

train = sess.upload_data(
    path = 'train.csv', 
    bucket = bucket,
    key_prefix = prefix
)

In [ ]:
%%writefile script.py


